Florent Giauna (AMSD) et Zewei Lin (MLSD)

Appentissage supervisé pour des données avec classes déséquilibrées

Séance 3 - Prédiction de churn, Partie II

In [1]:
#Librairies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Pré-traitement
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler

#Sélection de modèles
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV

#Modèles
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

#Métriques
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score

# Dataset : Credit fraud

In [2]:
#Chargement des données 
df = pd.read_csv('data/creditcard_v2.csv')

#Séparation du dataset en jeux d'entraînement et de test
X = df.drop('Class', axis=1)
y = df['Class']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=7, shuffle=True, stratify=y)

In [3]:
#Liste des variables
var_list = list(X_train)
    
#Redimensionnement des variables quantitatives
scaler = RobustScaler()

for var in var_list:
    X_train[[var]] = scaler.fit_transform(X_train[[var]])
    X_test[[var]] = scaler.transform(X_test[[var]])

Les variables contenant de nombreux outliers RobustScaler est plus approprié que StandardScaler et MinMaxScaler.

La courbe ROC (Receiver operating characteristic) permet de mesurer la performance d'un classifieur en contrebalançant la proportion de vrais positifs correctement prédits (recall ou sensitivity) par la proportion de vrais négatifs correctement prédits (specificity ou inverse de la précision). Le meilleur algorithme maximise l'aire sous la courbe ROC: l'AUC. 

Cependant utiliser la courbe ROC et l'AUC lorsque le jeu de données est déséquilibré pose problème (cf. Saito, T., & Rehmsmeier, M. (2015). The precision-recall plot is more informative than the ROC plot when evaluating binary classifiers on imbalanced datasets. PloS one, 10(3), e0118432. https://doi.org/10.1371/journal.pone.0118432). La courbe Precision-Recall (PR) et le PR AUC sont plus adaptés. Il s'agit de la moyenne des précisions sur les classes, calculée à chaque seuil de recall.

Dans notre cas, la PR AUC (average precision) est donc une meilleure mesure pour l'instant (avant d'essayer des techniques d'upsampling et de downsampling).

## Random Forest

In [4]:
#Instanciation du modèle RandomForest
clf_rf = RandomForestClassifier()

#Création de la grille de paramètres à tester
param_grid = {'n_estimators': [25, 50],
              'criterion': ['gini', 'entropy'],
              'max_depth': [13,15,17],
              'class_weight': ['balanced'],
              'random_state': [7]}

grid = GridSearchCV(estimator=clf_rf, 
                    param_grid=param_grid,
                    scoring='average_precision',
                    return_train_score=True,
                    cv=5)

grid.fit(X_train, y_train)

print("Meilleure configuration:", grid.best_params_, 
      "\nAverage precision (PR AUC):", grid.best_score_)

Meilleure configuration: {'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 15, 'n_estimators': 50, 'random_state': 7} 
Average precision (PR AUC): 0.8427836737342986


## XGBoost

Le paramètre 'scale_pos_weight' permet de donner plus de poids au gradient de la classe positive. Les erreurs commises par le modèle sur la classe positive sont considérées comme plus importantes. Cela peut l'aider à obtenir de meilleures performances sur la classe minoritaire. 

La documentation de XGBoost suggère de fixer la valeur du poids en divisant le nombre d'exemples de la classe majoritaire par le nombre d'exemples de la classe minoritaire (dans le jeu d'entraînement).

In [12]:
#Instanciation du modèle XGBoost
clf_xgb = XGBClassifier()

#Création de la grille de paramètres à tester
param_grid = {'learning_rate': [0.1, 0.3],
              'max_depth': [3,5,7],
              'n_estimators': [25, 50],
              'reg_alpha': [0.001,0.01],
              'reg_lambda': [0.001,0.01],
              'scale_pos_weight': [y_train.value_counts()[0] /
                                   y_train.value_counts()[1]],
              'random_state': [7]}

grid = GridSearchCV(estimator=clf_xgb, 
                    param_grid=param_grid,
                    scoring='average_precision',
                    return_train_score=True,
                    cv=5)

grid.fit(X_train, y_train)

print("Meilleure configuration:", grid.best_params_, 
      "\nAverage precision (PR AUC):", grid.best_score_)

Meilleure configuration: {'learning_rate': 0.3, 'max_depth': 5, 'n_estimators': 50, 'random_state': 7, 'reg_alpha': 0.001, 'reg_lambda': 0.001, 'scale_pos_weight': 0.0017322412299792043} 
Average precision (PR AUC): 0.7525885963156458


## Meilleur modèle

XGBoost a des résultats un peu en dessous de RandomForest mais ce dernier prend beaucoup plus de temps (27 min pour 49 modèles contre 17 min pour 121 modèles avec XGBoost). XGBoost semble être une meilleure option.

In [40]:
#Instanciation du meilleur modèle
clf = XGBClassifier(learning_rate=0.3, 
                    max_depth=5, 
                    n_estimators=50, 
                    random_state=7,
                    reg_alpha=0.01, 
                    reg_lambda=0.01, 
                    scale_pos_weight=(y_train.value_counts()[0] / 
                                      y_train.value_counts()[1]))

clf.fit(X_train, y_train)

#Classification du jeu test
pred = cross_val_predict(clf, X_test, y_test, cv=5)

#Résultats
clf_results = cross_validate(clf, X_test, y_test, scoring='average_precision', cv=5)
print("\nAverage precision (PR AUC):", clf_results['test_score'].mean())


Average precision (PR AUC): 0.8207177563406534


In [41]:
#Matrice de confusion
print(confusion_matrix(y_test, pred))

[[56851    13]
 [   22    76]]


In [42]:
#Rapport de classification
print(classification_report(y_test, pred, target_names=["0", "1"]))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.85      0.78      0.81        98

    accuracy                           1.00     56962
   macro avg       0.93      0.89      0.91     56962
weighted avg       1.00      1.00      1.00     56962



Ce résultat est jusqu'à présent le meilleur que nous ayons obtenu sur ce jeu de données. La précision sur la classe minoritaire est désormais de 85%, mais le recall a diminué. Comparons avec le meilleur RandomForest.

In [44]:
#Instanciation du meilleur RandomForest
clf = RandomForestClassifier(class_weight='balanced', 
                             criterion='entropy', 
                             max_depth=15, 
                             n_estimators=50,
                             random_state=7)

clf.fit(X_train, y_train)

#Classification du jeu test
pred = cross_val_predict(clf, X_test, y_test, cv=5)

#Résultats
clf_results = cross_validate(clf, X_test, y_test, scoring='average_precision', cv=5)
print("\nAverage precision (PR AUC):", clf_results['test_score'].mean())


Average precision (PR AUC): 0.8251964392069787


In [45]:
#Matrice de confusion
print(confusion_matrix(y_test, pred))

[[56861     3]
 [   30    68]]


In [46]:
#Rapport de classification
print(classification_report(y_test, pred, target_names=["0", "1"]))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.96      0.69      0.80        98

    accuracy                           1.00     56962
   macro avg       0.98      0.85      0.90     56962
weighted avg       1.00      1.00      1.00     56962



La précision sur la classe minoritaire est de 96%, mais le recall et le F1 score sur cette classe sont en dessous de ceux obtenus avec XGBoost. En outre le modèle requiert deux fois plus de temps pour s'exécuter.

XGBoost est donc un meilleur choix mais les techniques de rééquilibrage doivent encore être testées avant de mettre ce modèle en production.

# Dataset : Bank marketing

In [8]:
#Chargement des données 
df = pd.read_csv('data/bank-additional-full_v2.csv')

In [9]:
#Séparation du dataset en jeux d'entraînement et de test
X = df.drop('y', axis=1)
y = df['y']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=7, shuffle=True, stratify=y.to_numpy())

In [10]:
#Liste des variables catégorielles nominales
var_nom = list(X_train.select_dtypes(['object']).columns)
var_nom += ['age', 'duration', 'pdays']

#Liste des variables quantitatives
var_quant = ['campaign', 'cons.conf.idx', 'cons.price.idx', 'emp.var.rate', 
             'euribor3m', 'nr.employed', 'previous']

In [11]:
#Encodage des variables nominales 
ohe = OneHotEncoder(sparse_output=False).set_output(transform="pandas")

for var in var_nom:
    ohe_train = ohe.fit_transform(X_train[[var]])
    X_train = pd.concat([X_train, ohe_train],axis=1).drop(columns=[var])
    ohe_test = ohe.transform(X_test[[var]])
    X_test = pd.concat([X_test, ohe_test],axis=1).drop(columns=[var])

In [13]:
#Redimensionnement des variables quantitatives
scaler = RobustScaler()

for var in var_quant:
    X_train[[var]] = scaler.fit_transform(X_train[[var]])
    X_test[[var]] = scaler.transform(X_test[[var]])

Les variables n'ayant pas de distribution gaussienne et le jeu de données contenant de nombreux outliers RobustScaler est plus approprié que StandardScaler et MinMaxScaler.

La courbe ROC (Receiver operating characteristic) permet de mesurer la performance d'un classifieur en contrebalançant la proportion de vrais positifs correctement prédits (recall ou sensitivity) par la proportion de vrais négatifs correctement prédits (specificity ou inverse de la précision). Le meilleur algorithme maximise l'aire sous la courbe ROC: l'AUC. 

Cependant utiliser la courbe ROC et l'AUC lorsque le jeu de données est déséquilibré pose problème (cf. Saito, T., & Rehmsmeier, M. (2015). The precision-recall plot is more informative than the ROC plot when evaluating binary classifiers on imbalanced datasets. PloS one, 10(3), e0118432. https://doi.org/10.1371/journal.pone.0118432). La courbe Precision-Recall (PR) et le PR AUC sont plus adaptés. Il s'agit de la moyenne des précisions sur les classes, calculée à chaque seuil de recall.

Dans notre cas, la PR AUC (average precision) est donc une meilleure mesure pour l'instant (avant d'essayer des techniques d'upsampling et de downsampling).

## Random Forest

In [15]:
#Instanciation du modèle RandomForest
clf_rf = RandomForestClassifier()

#Création de la grille de paramètres à tester
param_grid = {'n_estimators': [50,100],
              'criterion': ['gini', 'entropy'],
              'max_depth': [5,7,9,11,15,30],
              'class_weight': ['balanced'],
              'random_state': [7]}

grid = GridSearchCV(estimator=clf_rf, 
                    param_grid=param_grid,
                    scoring='average_precision',
                    return_train_score=True,
                    cv=5)

grid.fit(X_train, y_train)

print("Meilleure configuration:", grid.best_params_, 
      "\nAverage precision (PR AUC):", grid.best_score_)

Meilleure configuration: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 9, 'n_estimators': 100, 'random_state': 7} 
Average precision (PR AUC): 0.5823520663192963


## XGBoost

Le paramètre 'scale_pos_weight' permet de donner plus de poids au gradient de la classe positive. Les erreurs commises par le modèle sur la classe positive sont considérées comme plus importantes. Cela peut l'aider à obtenir de meilleures performances sur la classe minoritaire. 

La documentation de XGBoost suggère de fixer la valeur du poids en divisant le nombre d'exemples de la classe majoritaire par le nombre d'exemples de la classe minoritaire (dans le jeu d'entraînement).

In [66]:
#Instanciation du modèle XGBoost
clf_xgb = XGBClassifier()

#Création de la grille de paramètres à tester
param_grid = {'learning_rate': [0.3],
              'max_depth': [1,3,5,15,30],
              'n_estimators': [100],
              'reg_alpha': [0.01,0.1],
              'reg_lambda': [0.01,0.1],
              'scale_pos_weight': [y_train.value_counts()[0] /
                                   y_train.value_counts()[1]],
              'random_state': [7]}

grid = GridSearchCV(estimator=clf_xgb, 
                    param_grid=param_grid,
                    scoring='average_precision',
                    return_train_score=True,
                    cv=5)

grid.fit(X_train, y_train)

print("Meilleure configuration:", grid.best_params_, 
      "\nAverage precision (PR AUC):", grid.best_score_)

Meilleure configuration: {'learning_rate': 0.3, 'max_depth': 3, 'n_estimators': 100, 'random_state': 7, 'reg_alpha': 0.01, 'reg_lambda': 0.01, 'scale_pos_weight': 7.876616379310345} 
Average precision (PR AUC): 0.5925114736928816


## Meilleur modèle

RandomForest a des résultats un peu en dessous de XGBoost mais ce dernier prend beaucoup plus de temps (6 min pour 100 modèles contre 2 min pour 100 modèles avec RandomForest). RandomForest semble être une meilleure option.

In [17]:
#Instanciation du meilleur modèle
clf = RandomForestClassifier(class_weight='balanced', 
                             criterion='entropy', 
                             max_depth=9, 
                             n_estimators=100,
                             random_state=7)

clf.fit(X_train, y_train)

#Classification du jeu test
pred = cross_val_predict(clf, X_test, y_test, cv=5)

#Résultats
clf_results = cross_validate(clf, X_test, y_test, scoring='average_precision', cv=5)
print("\nAverage precision (PR AUC):", clf_results['test_score'].mean())


Average precision (PR AUC): 0.5671260333037841


In [18]:
#Matrice de confusion
print(confusion_matrix(y_test, pred))

[[6111 1199]
 [ 160  768]]


In [85]:
#Rapport de classification
print(classification_report(y_test, pred, target_names=["0", "1"]))

              precision    recall  f1-score   support

           0       0.97      0.84      0.90      7310
           1       0.39      0.83      0.53       928

    accuracy                           0.84      8238
   macro avg       0.68      0.83      0.72      8238
weighted avg       0.91      0.84      0.86      8238



Ce résultat est jusqu'à présent le meilleur que nous ayons obtenu sur ce jeu de données. Mais il y a encore énormément de faux négatifs et la précision sur la classe minoritaire n'a pas beaucoup augmenté (39% contre 34% précédemment), mais il y a encore énormément de faux négatifs. Comparons avec le meilleur XGBoost.

In [26]:
#Instanciation du meilleur XGBoost
clf = XGBClassifier(learning_rate=0.3, 
                    max_depth=3, 
                    n_estimators=100, 
                    random_state=7,
                    reg_alpha=0.01, 
                    reg_lambda=0.01, 
                    scale_pos_weight=(y_train.value_counts()[0] / 
                                      y_train.value_counts()[1]))

clf.fit(X_train, y_train)

#Classification du jeu test
pred = cross_val_predict(clf, X_test, y_test, cv=5)

#Résultats
clf_results = cross_validate(clf, X_test, y_test, scoring='average_precision', cv=5)
print("\nAverage precision (PR AUC):", clf_results['test_score'].mean())


Average precision (PR AUC): 0.5511998973945706


In [27]:
#Matrice de confusion
print(confusion_matrix(y_test, pred))

[[5961 1349]
 [ 162  766]]


In [28]:
#Rapport de classification
print(classification_report(y_test, pred, target_names=["0", "1"]))

              precision    recall  f1-score   support

           0       0.97      0.82      0.89      7310
           1       0.36      0.83      0.50       928

    accuracy                           0.82      8238
   macro avg       0.67      0.82      0.70      8238
weighted avg       0.90      0.82      0.84      8238



Les résultats sont moins bons alors que le modèle est plus long à s'exécuter.

RandomForest est donc un meilleur choix mais les techniques de rééquilibrage doivent encore être testées avant de mettre ce modèle en production.

# Dataset : Employee attrition

In [29]:
#Chargement des données 
df = pd.read_csv('data/whole_data_v2.csv')

In [30]:
#Liste des variables quantitatives
var_quant = ['Age', 'DistanceFromHome', 'MonthlyIncome', 'NumCompaniesWorked', 
            'PercentSalaryHike', 'TotalWorkingYears', 'TrainingTimesLastYear', 
            'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager']

#Liste des variables qualitatives
var_cat = ['JobInvolvement', 'PerformanceRating', 'EnvironmentSatisfaction', 'JobSatisfaction',
           'WorkLifeBalance', 'BusinessTravel', 'Department', 'Education', 
           'EducationField', 'Gender', 'MaritalStatus', 'JobLevel', 'JobRole', 'StockOptionLevel']

#Liste des variables qualitatives ordinales
var_ord = ['JobInvolvement', 'PerformanceRating', 'EnvironmentSatisfaction', 'JobSatisfaction',
           'WorkLifeBalance', 'BusinessTravel', 'JobLevel', 'StockOptionLevel']

#Liste des variables qualitatives nominales
var_nom = ['Department', 'Education', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus']

In [31]:
#Séparation du dataset en jeux d'entraînement et de test
X = df.drop('Attrition', axis=1)
y = df['Attrition']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=7, shuffle=True, stratify=y.to_numpy())

In [32]:
#Encodage des variables nominales 
ohe = OneHotEncoder(sparse_output=False).set_output(transform="pandas")

for var in var_nom:
    ohe_train = ohe.fit_transform(X_train[[var]])
    X_train = pd.concat([X_train, ohe_train],axis=1).drop(columns=[var])
    ohe_test = ohe.transform(X_test[[var]])
    X_test = pd.concat([X_test, ohe_test],axis=1).drop(columns=[var])

#Encodage des variables ordinales
encoder = OrdinalEncoder(categories=[[0,1,2,3,4,5]], 
                         handle_unknown='use_encoded_value',
                         unknown_value=99)

for var in var_ord:
    X_train[[var]] = encoder.fit_transform(X_train[[var]])
    X_test[[var]] = encoder.transform(X_test[[var]])

In [33]:
#Redimensionnement des variables quantitatives
scaler = RobustScaler()

for var in var_quant:
    X_train[[var]] = scaler.fit_transform(X_train[[var]])
    X_test[[var]] = scaler.transform(X_test[[var]])

Les variables n'ayant pas de distribution gaussienne (hormis Age) et le jeu de données contenant de nombreux outliers RobustScaler est plus approprié que StandardScaler et MinMaxScaler.

La courbe ROC (Receiver operating characteristic) permet de mesurer la performance d'un classifieur en contrebalançant la proportion de vrais positifs correctement prédits (recall ou sensitivity) par la proportion de vrais négatifs correctement prédits (specificity ou inverse de la précision). Le meilleur algorithme maximise l'aire sous la courbe ROC: l'AUC. 

Cependant utiliser la courbe ROC et l'AUC lorsque le jeu de données est déséquilibré pose problème (cf. Saito, T., & Rehmsmeier, M. (2015). The precision-recall plot is more informative than the ROC plot when evaluating binary classifiers on imbalanced datasets. PloS one, 10(3), e0118432. https://doi.org/10.1371/journal.pone.0118432). La courbe Precision-Recall (PR) et le PR AUC sont plus adaptés. Il s'agit de la moyenne des précisions sur les classes, calculée à chaque seuil de recall.

Dans notre cas, la PR AUC (average precision) est donc une meilleure mesure pour l'instant (avant d'essayer des techniques d'upsampling et de downsampling).

## Random Forest

In [49]:
#Instanciation du modèle RandomForest
clf_rf = RandomForestClassifier()

#Création de la grille de paramètres à tester
param_grid = {'n_estimators': [50,100],
              'criterion': ['gini', 'entropy'],
              'max_depth': [1,5,10,15,20,25,35,46],
              'class_weight': ['balanced'],
              'random_state': [7]}

grid = GridSearchCV(estimator=clf_rf, 
                    param_grid=param_grid,
                    scoring='average_precision',
                    return_train_score=True,
                    cv=5)

grid.fit(X_train, y_train)

print("Meilleure configuration:", grid.best_params_, 
      "\nAverage precision (PR AUC):", grid.best_score_)

Meilleure configuration: {'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 20, 'n_estimators': 100, 'random_state': 7} 
Average precision (PR AUC): 0.9755055076740135


## XGBoost

Le paramètre 'scale_pos_weight' permet de donner plus de poids au gradient de la classe positive. Les erreurs commises par le modèle sur la classe positive sont considérées comme plus importantes. Cela peut l'aider à obtenir de meilleures performances sur la classe minoritaire. 

La documentation de XGBoost suggère de fixer la valeur du poids en divisant le nombre d'exemples de la classe majoritaire par le nombre d'exemples de la classe minoritaire (dans le jeu d'entraînement).

In [54]:
#Instanciation du modèle XGBoost
clf_xgb = XGBClassifier()

#Création de la grille de paramètres à tester
param_grid = {'learning_rate': [0.3],
              'max_depth': [5,10,15,20,25,35,46],
              'n_estimators': [100],
              'reg_alpha': [0.01,0.1],
              'reg_lambda': [0.01,0.1],
              'scale_pos_weight': [y_train.value_counts()[0] /
                                   y_train.value_counts()[1]],
              'random_state': [7]}

grid = GridSearchCV(estimator=clf_xgb, 
                    param_grid=param_grid,
                    scoring='average_precision',
                    return_train_score=True,
                    cv=5)

grid.fit(X_train, y_train)

print("Meilleure configuration:", grid.best_params_, 
      "\nAverage precision (PR AUC):", grid.best_score_)

Meilleure configuration: {'learning_rate': 0.3, 'max_depth': 15, 'n_estimators': 100, 'random_state': 7, 'reg_alpha': 0.01, 'reg_lambda': 0.01, 'scale_pos_weight': 5.214539007092198} 
Average precision (PR AUC): 0.96372038264088


## Meilleur modèle

RandomForest a de meilleurs résultats que XGBoost et prend moins de temps pour s'exécuter (27 s pour 144 modèles contre 6 min pour 100 modèles contre 53 s pour 169 modèles avec XGBoost). Mais RandomForest semble un peu plus complexe que XGBoost (profondeur de 20 contre 15). Comparons leurs résultats sur le jeu test.

In [56]:
#Instanciation du meilleur RandomForest
clf = RandomForestClassifier(class_weight='balanced', 
                             criterion='entropy', 
                             max_depth=20,
                             n_estimators=100,
                             random_state=7)
clf.fit(X_train, y_train)

#Classification du jeu test
pred = cross_val_predict(clf, X_test, y_test, cv=5)

#Résultats
clf_results = cross_validate(clf, X_test, y_test, scoring='average_precision', cv=5)
print("\nAverage precision (PR AUC):", clf_results['test_score'].mean())


Average precision (PR AUC): 0.6803778784195469


In [59]:
#Matrice de confusion
print(confusion_matrix(y_test, pred))

[[730   6]
 [ 89  52]]


In [60]:
#Rapport de classification
print(classification_report(y_test, pred, target_names=["0", "1"]))

              precision    recall  f1-score   support

           0       0.89      0.99      0.94       736
           1       0.90      0.37      0.52       141

    accuracy                           0.89       877
   macro avg       0.89      0.68      0.73       877
weighted avg       0.89      0.89      0.87       877



Ce résultat est jusqu'à présent le meilleur que nous ayons obtenu sur ce jeu de données. La précision sur la classe minoritaire est désormais de 90%, au détriment du recall (passé de 53% à 37%). Il y a plus de faux positifs mais beaucoup moins de faux négatifs, ce qui est plus souhaitable dans notre cas. Comparons avec le meilleur XGBoost.

In [61]:
#Instanciation du meilleur XGBoost
clf = XGBClassifier(learning_rate=0.3, 
                    max_depth=15, 
                    n_estimators=100, 
                    random_state=7,
                    reg_alpha=0.01, 
                    reg_lambda=0.01, 
                    scale_pos_weight=(y_train.value_counts()[0] / 
                                      y_train.value_counts()[1]))

clf.fit(X_train, y_train)

#Classification du jeu test
pred = cross_val_predict(clf, X_test, y_test, cv=5)

#Résultats
clf_results = cross_validate(clf, X_test, y_test, scoring='average_precision', cv=5)
print("\nAverage precision (PR AUC):", clf_results['test_score'].mean())


Average precision (PR AUC): 0.6449355663065015


In [62]:
#Matrice de confusion
print(confusion_matrix(y_test, pred))

[[699  37]
 [ 69  72]]


In [63]:
#Rapport de classification
print(classification_report(y_test, pred, target_names=["0", "1"]))

              precision    recall  f1-score   support

           0       0.91      0.95      0.93       736
           1       0.66      0.51      0.58       141

    accuracy                           0.88       877
   macro avg       0.79      0.73      0.75       877
weighted avg       0.87      0.88      0.87       877



Le F1 score macro average est un peu meilleur. Ce modèle a un meilleur recall sur la classe minoritaire mais sa précision sur cette classe est très en dessous du RandomForest. Il génère plus de faux négatifs. 

RandomForest semble être un meilleur choix mais les techniques de rééquilibrage doivent encore être testées avant de mettre ce modèle en production.